In [2]:
## This file is meant to be run only once
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd
import os.path
#from PIL import Image
import rasterio
import math

# read in all of your files #TODO make sure these match your file path
os.chdir("/oak/stanford/groups/omramom/group_members/aminaly/mountain_biodiversity")
os.chdir("/Users/aminaly/Box Sync/mountain_biodiversity")

In [ ]:
#### read in and combine all of the wdpa files. ONLY NEEDS TO RUN ONCE

## read in all the wdpa files
wdpa0 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_0/WDPA_May2021_Public_shp-polygons.shp")
wdpa1 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_1/WDPA_May2021_Public_shp-polygons.shp")
wdpa2 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_2/WDPA_May2021_Public_shp-polygons.shp")
wdpa3 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_0/WDPA_May2021_Public_shp-points.shp")
wdpa4 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_1/WDPA_May2021_Public_shp-points.shp")
wdpa5 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_2/WDPA_May2021_Public_shp-points.shp")

#combine the poly and point dataframes
wdpa_poly = gpd.GeoDataFrame(pd.concat([wdpa0, wdpa1, wdpa2], sort = True))
wdpa_point = gpd.GeoDataFrame(pd.concat([wdpa3, wdpa4, wdpa5], sort = True))

wdpa_poly.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}

#save these out so we don't have to re-run again
wdpa_poly.to_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-polygons.shp", driver='ESRI Shapefile')
wdpa_point.to_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-points.shp", driver='ESRI Shapefile')

In [3]:
#### If you don't need to run the above cell, here are the final poly and points to work qith
wdpa_poly = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-polygons.shp", driver='ESRI Shapefile')
wdpa_point = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-points.shp", driver='ESRI Shapefile')


In [4]:
#### Take in points and create buffers around them based on reported area

## Add buffers to the point based on their area
#wdpa_point.crs = {'init': 'epsg:3763', 'no_defs': True}
#wdpa_point.to_crs({'init': 'epsg:3763'})

## calculate the radius
wdpa_point['radius'] = np.sqrt(wdpa_point.REP_AREA / np.pi )

## remove entries with 0 reported area/radius
wdpa_point.drop(wdpa_point[wdpa_point['radius'] <= 0].index, inplace = True)

## create the buffers
wdpa_point['geometry'] = wdpa_point.geometry.buffer(wdpa_point.radius)

# convert projection back
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:4326'})

,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,SUPP_INFO,CONS_OBJ,geometry,radius
0,4195.0,4195,1,Queen Elizabeth,Queen Elizabeth,National Park,National Park,National,II,Not Applicable,...,Not Reported,Not Reported,983,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((162.06268 -9.53330, 162.05371 -9.715...",1.862680
1,4362.0,4362,1,Kolombangara Forest Reserve,Kolombangara Forest Reserve,Controlled Forest,Controlled Forest,National,Not Assigned,Not Applicable,...,Draft,State Verified,1932,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((165.06215 -8.00000, 165.02373 -8.782...",7.978846
2,316912.0,316912,1,Naro - Terrestrial,Naro - Terrestrial,Controlled Forest,Controlled Forest,Not Applicable,Not Assigned,Not Applicable,...,Not Reported,State Verified,1932,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((161.55758 -9.53330, 161.55104 -9.666...",1.357577
3,316913.0,316913,1,Komarindi,Komarindi,Conservation Area,Conservation Area,National,V,Not Applicable,...,Not Reported,Not Reported,983,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((167.83797 -9.00000, 167.80023 -9.768...",7.837972
6,555547628.0,555547628,1,Pipa/Kororo (Marovo),Pipa/Kororo (Marovo),Marine Protected Area/Tabu,Marine Protected Area/Tabu,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,1686,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((159.02106 -8.22235, 159.01498 -8.346...",1.261566
7,555547629.0,555547629,1,Rendova Harbor,Rendova Harbor,Marine Protected Area/Tabu,Marine Protected Area/Tabu,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,1686,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((158.46171 -8.40000, 158.45628 -8.510...",1.128379
8,555547630.0,555547630,1,Opele,Opele,Marine Protected Area/Tabu,Marine Protected Area/Tabu,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,1686,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((160.28957 -8.89365, 160.28624 -8.961...",0.690988
9,555547631.0,555547631,1,Tobo,Tobo,Marine Protected Area/Tabu,Marine Protected Area/Tabu,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,1686,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((160.28957 -8.89365, 160.28624 -8.961...",0.690988
10,555547632.0,555547632,1,Sasavele/NB,Sasavele/NB,Marine Protected Area/Tabu,Marine Protected Area/Tabu,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,1686,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((158.32721 -8.30001, 158.32250 -8.395...",0.977205
11,555547633.0,555547633,1,Koqu Rua,Koqu Rua,Marine Protected Area,Marine Protected Area,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,1686,Not Reported,SLB,SLB,Not Applicable,Not Applicable,"POLYGON ((157.43922 -8.19761, 157.43759 -8.230...",0.337855


In [49]:
#### Identify all the overlaps, dissolve, and combine with non-overlaps
overlaps_poly = wdpa_point.intersects(wdpa_poly)
overlaps_poly = overlaps_poly.reset_index().drop(columns = 'index')

# subset poly and point based on the identified overlaps 
overlaps_poly_sh = wdpa_poly[overlaps_poly]
nonoverlaps_poly_sh = wdpa_poly[~overlaps_poly]

# combine the overlapping polygons with the points 
overlap_poly_point = gpd.GeoDataFrame(pd.concat([overlaps_poly_sh, wdpa_point], sort = True))

#dissolve the overlapping polygons with all the points
dissolved_overlaps = overlap_poly_point.dissolve(by='WDPAID')

# concat new dissolved layer, with the nonoverlapping polygons
wdpa_final = gpd.GeoDataFrame(pd.concat([nonoverlaps_poly_sh, dissolved_overlaps], sort = True))

#make sure the CRS is correct
wdpa_final.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_final.to_crs({'init': 'epsg:4326'})

#save out
wdpa_final.to_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_flattened.shp", driver='ESRI Shapefile'))


In [67]:
wrld_cntries = ['CHE', 'DEU', 'KOR', 'GHA']
wdpa_clipped = wdpa_final[wdpa_final['ISO3'].isin(wrld_cntries)]

wdpa_final.to_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/clipped_WDPA_May2021_Public_flattened.shp", driver='ESRI Shapefile')